# Functions

In [13]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq

In [5]:
# --- Black-Scholes Formula ---
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# --- Monte Carlo Simulation ---
def monte_carlo_call(S, K, T, r, sigma, n_sim=100000):
    Z = np.random.randn(n_sim)
    ST = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    payoff = np.maximum(ST - K, 0)
    return np.exp(-r * T) * np.mean(payoff)

# --- FDM (Crank-Nicolson) ---
def fdm_call(S0, K, T, r, sigma, S_max=11000, M=1000, N=100):
    dt = T / M
    dS = S_max / N
    S = np.linspace(0, S_max, N + 1)
    V = np.maximum(S - K, 0)
    alpha = 0.25 * dt * (sigma ** 2 * (np.arange(N + 1) ** 2) - r * np.arange(N + 1))
    beta = -dt * 0.5 * (sigma ** 2 * (np.arange(N + 1) ** 2) + r)
    gamma = 0.25 * dt * (sigma ** 2 * (np.arange(N + 1) ** 2) + r * np.arange(N + 1))
    A = np.zeros((N - 1, N - 1))
    B = np.zeros((N - 1, N - 1))
    for i in range(N - 1):
        if i > 0:
            A[i, i - 1] = -alpha[i + 1]
            B[i, i - 1] = alpha[i + 1]
        A[i, i] = 1 - beta[i + 1]
        B[i, i] = 1 + beta[i + 1]
        if i < N - 2:
            A[i, i + 1] = -gamma[i + 1]
            B[i, i + 1] = gamma[i + 1]
    V_new = np.zeros(N + 1)
    for j in range(M):
        rhs = B @ V[1:N]
        rhs[0] += alpha[1] * (V[0] + V_new[0])
        rhs[-1] += gamma[N - 1] * (V[N] + V_new[N])
        V_new[1:N] = np.linalg.solve(A, rhs)
        V_new[0] = 0
        V_new[N] = S_max - K * np.exp(-r * (T - (j + 1) * dt))
        V[:] = V_new[:]
    # Interpolate to get price at S0
    return np.interp(S0, S, V)

# --- FEM (1D) ---
def fem_call(S0, K, T, r, sigma, S_max=11000, N=100, M=100):
    """
    1D FEM for Black-Scholes (European call, implicit Euler in time, linear elements)
    S0: initial asset price
    K: strike price
    T: time to maturity
    r: risk-free rate
    sigma: volatility
    S_max: max asset price in grid
    N: number of spatial elements (N+1 nodes)
    M: number of time steps
    """
    # Spatial grid
    S = np.linspace(0, S_max, N + 1)
    dS = S[1] - S[0]
    dt = T / M

    # Initial condition (payoff at maturity)
    V = np.maximum(S - K, 0)

    # Precompute local element matrices
    # For each element [S_i, S_{i+1}]
    # Linear basis: phi_0(x), phi_1(x)
    # Local mass and stiffness matrices
    M_loc = (dS / 6) * np.array([[2, 1], [1, 2]])
    K_loc = (1 / dS) * np.array([[1, -1], [-1, 1]])

    # Time stepping (backward in time)
    for n in range(M):
        # Assemble global matrices
        A = np.zeros((N + 1, N + 1))
        b = np.zeros(N + 1)

        for i in range(N):
            S_mid = 0.5 * (S[i] + S[i + 1])
            sigma2S2 = sigma ** 2 * S_mid ** 2
            rS = r * S_mid

            # Local matrices for this element
            # Diffusion term: (1/2) * sigma^2 * S^2 * d2V/dS2
            K_e = 0.5 * sigma2S2 * K_loc
            # Convection term: r * S * dV/dS
            C_e = rS * np.array([[ -1, 1], [ -1, 1]]) / 2
            # Reaction term: -r * V
            R_e = -r * M_loc

            # Time stepping: (M + dt*(K + C + R)) V^{n+1} = M V^n
            A_e = M_loc + dt * (K_e + C_e + R_e)
            b_e = M_loc @ V[i:i+2]

            # Assemble into global matrix
            A[i:i+2, i:i+2] += A_e
            b[i:i+2] += b_e

        # Boundary conditions
        # V(0) = 0
        A[0, :] = 0
        A[0, 0] = 1
        b[0] = 0
        # V(S_max) = S_max - K*exp(-r*t)
        t = T - (n + 1) * dt
        A[-1, :] = 0
        A[-1, -1] = 1
        b[-1] = S_max - K * np.exp(-r * t)

        # Solve linear system
        V = np.linalg.solve(A, b)

    # Interpolate to get price at S0
    return np.interp(S0, S, V)

In [15]:
# --- Implied Volatility Estimation ---
def implied_volatility(method_func, S0, K, T, r, market_price, sigma_bounds=(0.01, 10.0), **kwargs):
    def objective(sigma):
        return method_func(S0, K, T, r, sigma, **kwargs) - market_price
    try:
        return brentq(objective, *sigma_bounds)
    except ValueError:
        return np.nan

# Results

In [30]:
# --- Main: Data and Comparison ---
if __name__ == "__main__":
    strikes = np.array([5125, 5225, 5325, 5425, 5525, 5625, 5725, 5825])
    market_prices = np.array([475, 405, 340, 280.5, 226, 179.5, 139, 105])
    S0 = 5420.3
    r = 0.05
    T = 1/3
    avg_sigma = 0.182

    print("Strike\tMarket\tBS\tMC\tFDM\tFEM")
    for K, C_market in zip(strikes, market_prices):
        mc = monte_carlo_call(S0, K, T, r, avg_sigma)
        fdm = fdm_call(S0, K, T, r, avg_sigma)
        fem = fem_call(S0, K, T, r, avg_sigma)
        bs = black_scholes_call(S0, K, T, r, avg_sigma)
        print(f"{K}\t{C_market:.1f}\t{bs:.2f}\t{mc:.2f}\t{fdm:.2f}\t{fem:.2f}")

Strike	Market	BS	MC	FDM	FEM
5125	475.0	459.52	459.50	460.17	385.14
5225	405.0	389.94	391.13	390.67	321.07
5325	340.0	326.78	327.71	327.51	264.03
5425	280.5	270.36	269.60	271.02	214.09
5525	226.0	220.77	220.88	221.29	171.11
5625	179.5	177.90	177.32	178.23	134.76
5725	139.0	141.46	140.45	141.58	104.56
5825	105.0	110.99	110.10	111.20	80.19


In [27]:
strikes = np.array([5125, 5225, 5325, 5425, 5525, 5625, 5725, 5825])
market_prices = np.array([475, 405, 340, 280.5, 226, 179.5, 139, 105])
S0 = 5420.3
r = 0.05
T = 1/3

implied_vols_bs = []
implied_vols_mc = []
implied_vols_fdm = []
implied_vols_fem = []

for K, market_price in zip(strikes, market_prices):
    # Black-Scholes
    sigma_bs = implied_volatility(black_scholes_call, S0, K, T, r, market_price)
    implied_vols_bs.append(sigma_bs)
    # Monte Carlo
    sigma_mc = implied_volatility(monte_carlo_call, S0, K, T, r, market_price, n_sim=100000)
    implied_vols_mc.append(sigma_mc)
    # FDM (Crank-Nicolson)
    sigma_fdm = implied_volatility(fdm_call, S0, K, T, r, market_price)
    implied_vols_fdm.append(sigma_fdm)
    # FEM
    sigma_fem = implied_volatility(fem_call, S0, K, T, r, market_price)
    implied_vols_fem.append(sigma_fem)

# --- Results Table ---
df = pd.DataFrame({
    "Strike": strikes,
    "Market Price": market_prices,
    "Implied Vol (BS)": implied_vols_bs,
    "Implied Vol (MC)": implied_vols_mc,
    "Implied Vol (FDM)": implied_vols_fdm,
    "Implied Vol (FEM)": implied_vols_fem,
})

In [28]:
df.round(5)

,Strike,Market Price,Implied Vol (BS),Implied Vol (MC),Implied Vol (FDM),Implied Vol (FEM)
0,5125,475.0,0.19804,0.19818,0.19740,0.23681
1,5225,405.0,0.19599,0.19685,0.19535,0.23234
2,5325,340.0,0.19335,0.19292,0.19275,0.22757
3,5425,280.5,0.19029,0.19094,0.18977,0.22260
4,5525,226.0,0.18619,0.18631,0.18578,0.21692
5,5625,179.5,0.18329,0.18296,0.18303,0.21224
6,5725,139.0,0.17993,0.18004,0.17983,0.20733
7,5825,105.0,0.17661,0.17651,0.17640,0.20237


## Previous version

In [ ]:
# --- Main: Data and Comparison ---
if __name__ == "__main__":
    strikes = np.array([5125, 5225, 5325, 5425, 5525, 5625, 5725, 5825])
    market_prices = np.array([475, 405, 340, 280.5, 226, 179.5, 139, 105])
    S0 = 5420.3
    r = 0.05
    T = 1/3

    # Implied volatility for each strike
    implied_vols = []
    for K, C_market in zip(strikes, market_prices):
        iv = implied_volatility_call(C_market, S0, K, T, r)
        implied_vols.append(iv)
    implied_vols = np.array(implied_vols)
    print("Strike\tMarket\tImplied Vol")
    for K, C, iv in zip(strikes, market_prices, implied_vols):
        print(f"{K}\t{C}\t{iv:.4f}")

    avg_sigma = np.nanmean(implied_vols)
    print(f"\nAverage implied volatility: {avg_sigma:.4f}\n")

    print("Strike\tMarket\tMC\tFDM\tFEM\tBS")
    for K, C_market in zip(strikes, market_prices):
        mc = monte_carlo_call(S0, K, T, r, avg_sigma)
        fdm = fdm_call(S0, K, T, r, avg_sigma)
        fem = fem_call(S0, K, T, r, avg_sigma)
        bs = black_scholes_call(S0, K, T, r, avg_sigma)
        print(f"{K}\t{C_market:.1f}\t{mc:.2f}\t{fdm:.2f}\t{fem:.2f}\t{bs:.2f}")